### Creating and testing magnetic dipoles and grid system

In [74]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import time

from Modules.muon import Muon
from Modules.positron import Positron
import Modules.functions as func

from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit

# Setting graph defaults to avoid repetition on each plot
mpl.rcParams["axes.formatter.limits"] = -2, 2  # Sets xticks to use exponents
mpl.rcParams["axes.grid"] = True  # Turns grid on
mpl.rcParams["legend.loc"] = "best"  # Turns legend on and autoplaces it

### Dipole class
Creates a dipole of some strength, with a given orientation and location
The magnetic moment "m" is calculated by:
$m = [r cos(\theta), r sin(\theta)]$<br>
Where $r$ is the strength and $\theta$ is the angle from the x axis<br>

The method "get_mag_field()" returns the magnetic field due to the dipole at a given point in space.

In [75]:
class Dipole(object):
    """
    Creates dipole:
    location = (pos_x, pos_y)
    orientation and length = pole_separation
    """
    count = 0
    def __init__(self, orientation, location, strength):
        """
        Init object: orientation (degrees), location (array), strength (scalar in Tesla)
        """
        self.location = np.array(location)
        self.orientation_d = orientation
        self.orientation_r = np.deg2rad(orientation)
        self.strength = strength
        self.moment = np.array([strength * np.cos(self.orientation_r), strength * np.sin(self.orientation_r), 0])
        Dipole.count += 1

    def __repr__(self):
        """Sets string representation of the instance"""
        return ("Dipole object:  Location: {}\n\
                Orientation: {:.2f} degrees\n\
                Strength: {:.3e} T\n\
                Moment: {}".format(self.location, self.orientation_d, self.strength, self.moment))

    def get_mag_field(self, target):
        """Gets magnetic field at target location (x, y, z)"""
        # Check that coordinates are same dimension
        if not len(target)==len(self.location):
            raise ValueError("Target location and dipole location must be given in the same dimensions")
        mag_perm = 4*np.pi*10**-7  # Cancel constant terms to get mag_perm as only constant
        relative_loc = np.subtract(np.array(target), self.location)
        magnitude = get_mag(relative_loc)
        return (mag_perm/4*np.pi) * (
                (3*relative_loc*(np.dot(self.moment, relative_loc)) / (magnitude**5))
                - (self.moment / (magnitude**3))
                )

    def get_relative_loc(self, other):
        return other.location - self.location


### Grid class (WiP)
Creates a grid into which the dipoles can be placed


In [76]:
class Grid(object):
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.add_points(width, height)
    
    def __repr__(self):
        #widthheight = f"Width: {self.width}, Height: {self.height}"
        #rowscols = f"x points: {self.cols}, y points: {self.rows}"
        #return widthheight + "\n" + rowscols
        return str(self.points)
    
    def add_to_point(self, x, y, new_value):
        # Makes addition to preexisting cell
        cur_value = self.get_point(x, y)
        self.change_point([x, y], cur_value + new_value)
    
    def add_points(self, x_points, y_points):
        self.points = np.zeros([x_points, y_points], dtype=object)
    
    def convert_y(self, y):
        return int(self.height - 1 - y)
    
    def change_point(self, location, value):
        x, y = location
        y = self.convert_y(y)
        self.points[y][x] = value

    def all_coords(self):
        for x in range(self.width):
            for y in range(self.height):
                yield (x, y)
    
    def get_point(self, x, y):
        y = self.convert_y(y)
        return self.points[y][x]


g = Grid(10, 10)
#g.add_points(3, 3)
g.change_point([0, 2], "x")
temp = g.get_point(0, 2)
temp = "O"
g.change_point((0, 2), temp)
g.add_to_point(0, 2, "x")
print(g)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 ['Ox' 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


### General functions

In [77]:

def set_unit(vector):
    norm = np.linalg.norm(vector)
    if norm == 0: 
       return vector
    return vector / norm

def get_mag(vector):
    return np.sqrt(vector.dot(vector))

def set_dir_2d(vector):
    return np.arctan(vector[1]/vector[0])

In [78]:
def set_field_values(x_array, y_array, dipole_grid, resolution=10):
    # Loop over points in grid and get field at each point
    field_x = x_array[::resolution]
    field_y = y_array[::resolution]
    field = Grid(len(field_x), len(field_y))
    for i, x in enumerate(field_x):
        for j, y in enumerate(field_y):
            for coord in dipole_grid.all_coords():
                target_dipole = dipole_grid.get_point(*coord)
                field_to_add = target_dipole.get_mag_field(target=[x, y, 100e-6])
                field.add_to_point(i, j, field_to_add)
    return field

Nx, Ny = 101, 101
side_length = 2e-3
x_array = np.linspace(-side_length, side_length, Nx)
y_array = np.linspace(-side_length, side_length, Ny)



'\ndef set_field_values(x_array, y_array, dipole_dict, resolution=10):\n    # Loop over points in grid and get field at each point\n    field_x = np.zeros([len(x_array), len(y_array)])\n    field_y = np.zeros_like(field_x)\n    try:\n        for i, x in enumerate(x_array[::resolution]):\n            for j, y in enumerate(y_array[::resolution]):\n                for coord, dipole in dipole_dict.items():\n                    field_at_point = dipole.get_mag_field(target=[x, y, 100e-6])\n                    field_x[i][j] += field_at_point[0]\n                    field_y[i][j] += field_at_point[1]\n    # Catches errors without crashing\n    except TypeError as e:\n        print("======ERROR======")\n        print("i, x", i, x)\n        print("j, y", j, y)\n        print(field_at_point)\n        raise\n    return field_x, field_y\n'

### Function to create the dipole grid

In [93]:
def create_dipole_grid(x_array, y_array, strength=1e-3, dipole_spacing=40, buffer=20, random_angle=False, angle=0, field_res=1):
    """
    Creates a grid of dipoles
    Places a dipole every {dipole_spacing} on the x/y array with a buffer around the edges
    """
    start = time.time()
    dipole_x_pos = x_array[buffer:-buffer:dipole_spacing]
    dipole_y_pos = y_array[buffer:-buffer:dipole_spacing]

    dipole_grid = Grid(len(dipole_x_pos), len(dipole_y_pos))
    for i, x in enumerate(dipole_x_pos):
        for j, y in enumerate(dipole_y_pos):
            if random_angle:
                angle = np.random.randint(0, 361)
            print(i, j)
            dipole_grid.change_point((i, j), Dipole(orientation=angle,location=[x, y, 0],strength=strength))
    mid = time.time()
    print("Made {} dipoles in {:.3}s".format(Dipole.count, mid-start))
    return dipole_grid


#field = set_field_values(x_array, y_array, dipole_grid, resolution=field_res)
#Nx, Ny = 1000, 1000
#side_length = 2e-3
#x_array = np.linspace(-side_length, side_length, Nx)
#y_array = np.linspace(-side_length, side_length, Ny)
#d_grid = create_dipole_grid(x_array, y_array)


##### Creating dipole grids
Aligned and random orientations

In [94]:
Nx, Ny = 101, 101
side_length = 2e-3
x_array = np.linspace(-side_length, side_length, Nx)
y_array = np.linspace(-side_length, side_length, Ny)

dipoles_aligned = create_dipole_grid(x_array, y_array,
                                    dipole_spacing=10,
                                    random_angle=False,
                                    strength=1e-2,
                                    buffer=0)
dipoles_random = create_dipole_grid(x_array, y_array,
                                   dipole_spacing=10,
                                   random_angle=True,
                                   strength=1e-2,
                                   buffer=0)

print(dipoles_random)

Made 0 dipoles in 4.01e-05s
Made 0 dipoles in 2.19e-05s
[]


##### Calculating field strength for all points in grid

In [81]:
strength_aligned = np.zeros_like(field_x_aligned)
strength_aligned += np.sqrt(field_x_aligned**2 + field_y_aligned**2)

strength_random = np.zeros_like(field_x_aligned)
strength_random += np.sqrt(field_x_random**2 + field_y_random**2)

NameError: name 'field_x_aligned' is not defined

In [ ]:
plt.figure()


### Plotting the field lines

In [ ]:
fig1, ax1 = plt.subplots(1, 1, sharey=True)
fig2, ax2 = plt.subplots(1, 1, sharey=True)

# STREAMPLOT
aligned = ax1.streamplot(x_array, y_array,
               field_y_aligned, field_x_aligned,
               density=3)


random = ax2.streamplot(x_array, y_array, field_y_random, field_x_random,
               density=3)

# DIRECTIONAL ARROW SHOWING DIPOLE
for coord, dip in dipoles_aligned.items():
    x, y = dip.location[1], dip.location[0]
    angle = dip.orientation_d + 90
    ax1.quiver(x, y, dip.moment[1]*2, dip.moment[0]*2,
               angles=angle, color="r", alpha=0.7)
    
for coord, dip in dipoles_random.items():
    x, y = dip.location[1], dip.location[0]
    angle = dip.orientation_d + 90
    ax2.quiver(x, y, dip.moment[1]*2, dip.moment[0]*2,
               angles=angle, color="r", alpha=0.7)

# SETTING PLOT SIZE AND FORMAT
ax1.set_xlim(-side_length-(side_length/10), side_length+(side_length/10))
ax1.set_ylim(-side_length-(side_length/10), side_length+(side_length/10))
ax2.set_xlim(-side_length-(side_length/10), side_length+(side_length/10))
ax2.set_ylim(-side_length-(side_length/10), side_length+(side_length/10))
plt.show()

In [ ]:
plt.figure()
plt.imshow(strength_aligned, cmap="hot", norm=colors.LogNorm(vmin=strength_aligned.min(),vmax=strength_aligned.max()))

plt.figure()
plt.imshow(strength_random, cmap="hot", norm=colors.LogNorm(vmin=strength_random.min(),vmax=strength_random.max()))